
# Pytorch + HuggingFace 
# KoBART

## Datasets
- 지역 20곳 & 각 지역별 식당 40여개 리뷰 전체 크롤링 <br>
- 강릉, 건대, 경주, 광안리, 남포동, 부평, 사당, 서귀포, 서면, 속초, 송도, 연신내, 왕십리, 일산, 전주한옥마을, 제주, 판교, 해운대, 혜화, 홍대

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
- GPU로 돌려주세요

#Preparation

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 설치
!pip install transformers # HuggingFace transformers
!pip install pandas tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.5 MB/s 
     |████████████████████████████████| 7.6 MB 71.0 MB/s 
     |████████████████████████████████| 182 kB 99.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW, TextClassificationPipeline,BartForSequenceClassification
from tqdm.notebook import tqdm

In [ ]:
# 데이터셋 불러오기

train = pd.read_csv('urp_trte.csv')
#test = pd.read_csv('test_set.csv')

In [ ]:
train.head()

,리뷰,grade,라벨링
0,국물이 냄새도없고 담백하고 맛있었어요,3.5,6.0
1,뷰 맛집~ \n근데 파스타는 별로에요.,3.0,5.0
2,옛날맛에서 변했어요 감튀도 냉동싸구려고 햄버거 내용물이 많이 짜졌어요,1.5,2.0
3,생각했던 스페인요리와는 조금 다르네요. 그래도 직원분들이 친절하십니다.,3.0,5.0
4,마이크로 웨이팅 번호 부르고 어디 앉아라 계속 말하는통에 음식이 입으로 들어가는지 ...,2.0,3.0


In [ ]:
train = train.applymap(str)

In [ ]:
train.to_csv('train.csv')

In [ ]:
train= './train.csv' 
#test = './test_set.csv'

In [ ]:
# GPU 사용
device = torch.device("cuda")

# Dataset

In [ ]:
class ClassifyDataset(Dataset):
  
  def __init__(self, csv_file):
    # csv 읽어오기 & NaN 처리
    self.dataset = pd.read_csv(csv_file).dropna(axis=0) 
    # 중복 제거
    self.dataset.drop_duplicates(subset=['리뷰'], inplace=True)
    # pretrained model
    self.tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart") 

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:4].values # idx 행과 0,1,2 columns
    text = row[1]
    y = row[3]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt', # return pytorch tensors
        truncation=True, # reducing long sequences
        max_length=256,
        pad_to_max_length=True, # padding
        add_special_tokens=True # 문장 앞뒤로 special tocken - padding 부착
        )
    
    input_ids = inputs['input_ids'][0] 
    attention_mask = inputs['attention_mask'][0] # the position of the padded indices so that the model does not attend to them

    return input_ids, attention_mask, y

In [ ]:
train_dataset = ClassifyDataset(train)
#test_dataset = ClassifyDataset(test)

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

         Unnamed: 0         grade           라벨링
count  46469.000000  46469.000000  46469.000000
mean   28643.058749      4.083830      7.167660
std    16823.728864      1.264272      2.528544
min        0.000000      0.500000      0.000000
25%    14131.000000      3.500000      6.000000
50%    28457.000000      5.000000      9.000000
75%    43114.000000      5.000000      9.000000
max    58182.000000      5.000000      9.000000


# Create Model & Train

In [ ]:
model = BartForSequenceClassification.from_pretrained("hyunwoongko/kobart", num_labels=10).to(device)
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model,device=0) # gpu로 변환

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
epochs = 10
batch_size = 32

optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
losses = []
accuracies = []

for i in range(epochs): # epoch 10
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train() # forward

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader): # tqdm 진행상황 확인 
    optimizer.zero_grad()
  
    y_batch = y_batch.type(torch.LongTensor)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0] 
    loss = F.cross_entropy(y_pred, y_batch)

    loss.backward()
    optimizer.step() 

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total) 

  0%|          | 0/1453 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 172.77346980571747 Accuracy: tensor(0.4884, device='cuda:0')
Batch Loss: 327.2199162244797 Accuracy: tensor(0.5120, device='cuda:0')
Batch Loss: 483.5840928554535 Accuracy: tensor(0.5109, device='cuda:0')
Batch Loss: 627.3006881475449 Accuracy: tensor(0.5203, device='cuda:0')
Batch Loss: 763.306459903717 Accuracy: tensor(0.5289, device='cuda:0')
Batch Loss: 896.2375773191452 Accuracy: tensor(0.5351, device='cuda:0')
Batch Loss: 1025.2705301046371 Accuracy: tensor(0.5387, device='cuda:0')
Batch Loss: 1153.1205896139145 Accuracy: tensor(0.5441, device='cuda:0')
Batch Loss: 1275.640516281128 Accuracy: tensor(0.5490, device='cuda:0')
Batch Loss: 1400.6544035673141 Accuracy: tensor(0.5511, device='cuda:0')
Batch Loss: 1524.2641443610191 Accuracy: tensor(0.5540, device='cuda:0')
Batch Loss: 1650.085418999195 Accuracy: tensor(0.5559, device='cuda:0')
Batch Loss: 1771.9565557837486 Accuracy: tensor(0.5582, device='cuda:0')
Batch Loss: 1890.7064497470856 Accuracy: tensor(0.5607, dev

  0%|          | 0/1453 [00:00<?, ?it/s]

Batch Loss: 118.76717525720596 Accuracy: tensor(0.5997, device='cuda:0')
Batch Loss: 241.93586909770966 Accuracy: tensor(0.5873, device='cuda:0')
Batch Loss: 359.92610162496567 Accuracy: tensor(0.5866, device='cuda:0')
Batch Loss: 473.4243136048317 Accuracy: tensor(0.5918, device='cuda:0')
Batch Loss: 588.9584137797356 Accuracy: tensor(0.5944, device='cuda:0')
Batch Loss: 701.863941192627 Accuracy: tensor(0.5974, device='cuda:0')
Batch Loss: 815.8119184374809 Accuracy: tensor(0.5996, device='cuda:0')
Batch Loss: 931.7609326839447 Accuracy: tensor(0.5993, device='cuda:0')
Batch Loss: 1047.469330906868 Accuracy: tensor(0.5999, device='cuda:0')
Batch Loss: 1161.307995736599 Accuracy: tensor(0.6008, device='cuda:0')
Batch Loss: 1273.339461028576 Accuracy: tensor(0.6014, device='cuda:0')
Batch Loss: 1388.9662168622017 Accuracy: tensor(0.6013, device='cuda:0')
Batch Loss: 1502.3874411582947 Accuracy: tensor(0.6021, device='cuda:0')
Batch Loss: 1614.8623989224434 Accuracy: tensor(0.6030, devi

...

Batch Loss: 91.9375291466713 Accuracy: tensor(0.6712, device='cuda:0')
Batch Loss: 181.19578582048416 Accuracy: tensor(0.6697, device='cuda:0')
Batch Loss: 273.75620698928833 Accuracy: tensor(0.6690, device='cuda:0')
Batch Loss: 362.7075911462307 Accuracy: tensor(0.6723, device='cuda:0')
Batch Loss: 452.0702956020832 Accuracy: tensor(0.6751, device='cuda:0')
Batch Loss: 540.7291995584965 Accuracy: tensor(0.6765, device='cuda:0')
Batch Loss: 632.2072272598743 Accuracy: tensor(0.6753, device='cuda:0')
Batch Loss: 722.4622386991978 Accuracy: tensor(0.6755, device='cuda:0')
Batch Loss: 813.1411135494709 Accuracy: tensor(0.6750, device='cuda:0')
Batch Loss: 905.7942296564579 Accuracy: tensor(0.6733, device='cuda:0')
Batch Loss: 996.2766639888287 Accuracy: tensor(0.6736, device='cuda:0')
Batch Loss: 1087.5441435575485 Accuracy: tensor(0.6734, device='cuda:0')
Batch Loss: 1178.0030604302883 Accuracy: tensor(0.6732, device='cuda:0')
Batch Loss: 1268.4926408827305 Accuracy: tensor(0.6734, devic

  0%|          | 0/1453 [00:00<?, ?it/s]

Batch Loss: 83.04499736428261 Accuracy: tensor(0.7063, device='cuda:0')
Batch Loss: 168.96282720565796 Accuracy: tensor(0.6975, device='cuda:0')
Batch Loss: 254.0789339840412 Accuracy: tensor(0.6943, device='cuda:0')
Batch Loss: 341.782773822546 Accuracy: tensor(0.6910, device='cuda:0')
Batch Loss: 424.36909145116806 Accuracy: tensor(0.6929, device='cuda:0')
Batch Loss: 511.0546410083771 Accuracy: tensor(0.6917, device='cuda:0')
Batch Loss: 596.3339020609856 Accuracy: tensor(0.6916, device='cuda:0')
Batch Loss: 683.0875877141953 Accuracy: tensor(0.6912, device='cuda:0')
Batch Loss: 772.8972916603088 Accuracy: tensor(0.6888, device='cuda:0')
Batch Loss: 860.9740162789822 Accuracy: tensor(0.6884, device='cuda:0')
Batch Loss: 946.7221620380878 Accuracy: tensor(0.6888, device='cuda:0')
Batch Loss: 1032.9879477024078 Accuracy: tensor(0.6892, device='cuda:0')
Batch Loss: 1120.776375502348 Accuracy: tensor(0.6890, device='cuda:0')
Batch Loss: 1203.974799066782 Accuracy: tensor(0.6904, device=

  0%|          | 0/1453 [00:00<?, ?it/s]

Batch Loss: 80.79026579856873 Accuracy: tensor(0.7109, device='cuda:0')
Batch Loss: 158.27707996964455 Accuracy: tensor(0.7138, device='cuda:0')
Batch Loss: 240.20114240050316 Accuracy: tensor(0.7093, device='cuda:0')
Batch Loss: 322.1046358048916 Accuracy: tensor(0.7072, device='cuda:0')
Batch Loss: 404.69077813625336 Accuracy: tensor(0.7054, device='cuda:0')
Batch Loss: 488.43558019399643 Accuracy: tensor(0.7033, device='cuda:0')
Batch Loss: 571.6382765769958 Accuracy: tensor(0.7025, device='cuda:0')
Batch Loss: 654.3761929571629 Accuracy: tensor(0.7012, device='cuda:0')
Batch Loss: 732.5735674500465 Accuracy: tensor(0.7024, device='cuda:0')
Batch Loss: 813.1014070510864 Accuracy: tensor(0.7036, device='cuda:0')
Batch Loss: 895.8991026878357 Accuracy: tensor(0.7027, device='cuda:0')
Batch Loss: 976.368039816618 Accuracy: tensor(0.7032, device='cuda:0')
Batch Loss: 1059.0953296124935 Accuracy: tensor(0.7031, device='cuda:0')
Batch Loss: 1142.0490306913853 Accuracy: tensor(0.7026, devi

In [ ]:
losses, accuracies

([1954.6154416203499,
  1677.3795059919357,
  1607.93995064497,
  1549.5433587431908,
  1496.7983712553978,
  1437.8989264518023,
  1378.048902451992,
  1315.3123779594898,
  1252.0851845443249,
  1184.7807992696762],
 [tensor(0.5613, device='cuda:0'),
  tensor(0.6026, device='cuda:0'),
  tensor(0.6143, device='cuda:0'),
  tensor(0.6233, device='cuda:0'),
  tensor(0.6347, device='cuda:0'),
  tensor(0.6443, device='cuda:0'),
  tensor(0.6582, device='cuda:0'),
  tensor(0.6741, device='cuda:0'),
  tensor(0.6899, device='cuda:0'),
  tensor(0.7027, device='cuda:0')])

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "naver_map_kobart_final.pt") 

In [ ]:
# 모델 불러오기
model.load_state_dict(torch.load("./naver_map_model.pt"))

<All keys matched successfully>

# TEST

## Evaluation

In [ ]:
test1 = pd.read_csv('./산골농원.csv')
test2 = pd.read_csv('./유일닭강정.csv')
test3 = pd.read_csv('./채원 잣두부.막국수.csv')

In [ ]:
test1.drop('Unnamed: 0', axis=1, inplace=True)
test2.drop('Unnamed: 0', axis=1, inplace=True)
test3.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
test3.head()

,날짜,리뷰
0,7.10.일,진짜 진짜 맛있었어요 자극적이지 않고 슴슴한데 너무 맛있었어요!!!\n국수는 비빔이...
1,7.9.토,맛나요~
2,7.9.토,좋아요
3,7.3.일,👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 ...
4,7.1.금,좋아요


In [ ]:
# 리뷰 텍스트 predicted label(0,1) 확인

def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart") 
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=256
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# gradient 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
        
    logits = outputs[0] 
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits) 
    return result

## Test Result

In [ ]:
#test_pd = pd.read_csv('test_set.csv')

In [ ]:
score = [] # label - score

total_len = len(test3)

for cnt, review in enumerate(test3['리뷰']): 
  pred = sentiment_classifier(review) 
  score.append(pred) 
  
  if cnt % 100 == 0:
    print(cnt, '개 문장 분류 완료')

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0 개 문장 분류 완료
100 개 문장 분류 완료
200 개 문장 분류 완료
300 개 문장 분류 완료
400 개 문장 분류 완료


In [ ]:
score

[[{'label': 'LABEL_9', 'score': 0.9801589250564575}],
 [{'label': 'LABEL_6', 'score': 0.31675031781196594}],
 [{'label': 'LABEL_5', 'score': 0.4439931809902191}],
 [{'label': 'LABEL_9', 'score': 0.8971653580665588}],
 [{'label': 'LABEL_6', 'score': 0.3745245933532715}],
 [{'label': 'LABEL_6', 'score': 0.4508323669433594}],
 [{'label': 'LABEL_7', 'score': 0.2663297653198242}],
 [{'label': 'LABEL_9', 'score': 0.8534383773803711}],
 [{'label': 'LABEL_9', 'score': 0.28172099590301514}],
 [{'label': 'LABEL_9', 'score': 0.9884560108184814}],
 [{'label': 'LABEL_9', 'score': 0.4394458532333374}],
 [{'label': 'LABEL_7', 'score': 0.310428649187088}],
 [{'label': 'LABEL_9', 'score': 0.30976602435112}],
 [{'label': 'LABEL_5', 'score': 0.4754341244697571}],
 [{'label': 'LABEL_0', 'score': 0.5155913233757019}],
 [{'label': 'LABEL_5', 'score': 0.34171658754348755}],
 [{'label': 'LABEL_6', 'score': 0.3468468189239502}],
 [{'label': 'LABEL_0', 'score': 0.4911855459213257}],
 [{'label': 'LABEL_9', 'scor

In [ ]:
test3['predicted'] = 0 # label(예측 결과)
score_list = [] # score 

for i in range(len(score)): 
    test3['predicted'][i] = int(score[i][0].get('label')[-1])
    score_list.append(float(score[i][0].get('score')))

<ipython-input-43-9b7acf63ee31>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test3['predicted'][i] = int(score[i][0].get('label')[-1])


In [ ]:
test3

,날짜,리뷰,predicted
0,7.10.일,진짜 진짜 맛있었어요 자극적이지 않고 슴슴한데 너무 맛있었어요!!!\n국수는 비빔이...,9
1,7.9.토,맛나요~,6
2,7.9.토,좋아요,5
3,7.3.일,👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 ...,9
4,7.1.금,좋아요,6
...,...,...,...
455,18.12.31.월,몇번째방문인진 몰라도 올때마다 맛있게먹고갑니다.,9
456,18.12.31.월,가족들과 함께 갈만한곳을 검색끝에 예약하고 방문. 전체적으로 맛이 깔끔합니다 육수가...,9
457,18.12.31.월,식당 깔끔하고 맛도 정갈하니 잘먹었습니다. 우리 꼬맹이들이 다 잘먹어서 기분이 좋았...,9
458,18.12.30.일,다른 음식도 다 맛있지만 특히 메밀막국수가 정말 맛있어요! 막국수 먹고싶을 때마다 ...,9


In [ ]:
score_df = pd.DataFrame({'score':score_list})
final_test_pd = pd.concat([test3,score_df],axis=1)
final_test_pd

,날짜,리뷰,predicted,score
0,7.10.일,진짜 진짜 맛있었어요 자극적이지 않고 슴슴한데 너무 맛있었어요!!!\n국수는 비빔이...,9,0.980159
1,7.9.토,맛나요~,6,0.316750
2,7.9.토,좋아요,5,0.443993
3,7.3.일,👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 👍 ...,9,0.897165
4,7.1.금,좋아요,6,0.374525
...,...,...,...,...
455,18.12.31.월,몇번째방문인진 몰라도 올때마다 맛있게먹고갑니다.,9,0.536556
456,18.12.31.월,가족들과 함께 갈만한곳을 검색끝에 예약하고 방문. 전체적으로 맛이 깔끔합니다 육수가...,9,0.859935
457,18.12.31.월,식당 깔끔하고 맛도 정갈하니 잘먹었습니다. 우리 꼬맹이들이 다 잘먹어서 기분이 좋았...,9,0.985155
458,18.12.30.일,다른 음식도 다 맛있지만 특히 메밀막국수가 정말 맛있어요! 막국수 먹고싶을 때마다 ...,9,0.917939


In [ ]:
# 결과 저장
final_test_pd.to_csv("채원 잣두부.막국수_pred.csv", encoding = 'utf-8-sig', index=False) 